In [1]:
import os
import sys
import time
import cv2
from PIL import Image, ImageDraw
from utils import *
from darknet import Darknet

In [2]:
def detect(cfgfile, weightfile, videoFile):
    frame_stream = cv2.VideoCapture(videoFile)
    
    m = Darknet(cfgfile)

    m.print_network()
    m.load_weights(weightfile)

    num_classes = 80
    if num_classes == 20:
        namesfile = 'data/voc.names'
    elif num_classes == 80:
        namesfile = 'data/coco.names'
    else:
        namesfile = 'data/names'

    use_cuda = 1
    if use_cuda:
        m.cuda()
    
    
    is_valid, frame = frame_stream.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame)
    resized = frame.resize((m.width, m.height))
    frame_count = 0
    total_time = 0
    output_dir = videoFile[:-4]+'_detections'
    os.mkdir(output_dir)
    while(1):
        if not is_valid:
            break
        else:
            #for i in range(2):
            #    t_start = time.time()
            #    boxes = do_detect(m, resized, 0.5, 0.4, use_cuda)
            #    t_finish = time.time()
            t_start = time.time()
            boxes = do_detect(m, resized, 0.5, 0.4, use_cuda)
            t_finish = time.time()
            #print("curr time: " + str(t_finish-t_start))
            total_time += t_finish-t_start
            class_names = load_class_names(namesfile)
            classified_frame = output_dir + "/output_" + str(frame_count) + ".jpg"
            plot_boxes(resized, boxes, classified_frame, class_names)
            is_valid, frame = frame_stream.read()
            if not is_valid:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            resized = frame.resize((m.width, m.height))
        frame_count += 1
    
    avgtime_per_frame = total_time/frame_count
    time_elapsed_mins = (total_time/60) - ((total_time/60) % 1)
    time_elapsed_secs = ((total_time/60)%1)*60
    print("---------yolo3" + str(videoFile) + " frames classified-----------")
    print("Average computation time per frame: " + str(avgtime_per_frame))
    print("Completed. Total time for computation: " + str(time_elapsed_mins) + " minutes, " + str(time_elapsed_secs) + " seconds.")
    

if __name__ == '__main__':
    for veh_cam in ['camera_1.mp4', 'camera_2.mp4', 'camera_5.mp4', 'camera_6.mp4']:
        detect('cfg/yolov3.cfg', 'yolov3.weights', veh_cam)

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32
    1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64
    2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32
    3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64
    4 shortcut 1
    5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128
    6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
    7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    8 shortcut 5
    9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
   10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
   11 shortcut 8
   12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x  52 x 256
   13 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128
   14 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
   15 shortcut 12
   16 conv    128  1 x 1

/home/dfpazr/Documents/ECE/ECE285/ece285-final-project/yolo3/utils.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_confs = torch.nn.Softmax()(Variable(output[5:5+num_classes].transpose(0,1))).data


---------yolo3camera_1.mp4 frames classified-----------
Average computation time per frame: 0.07218461484501401
Completed. Total time for computation: 5.0 minutes, 42.80473589897157 seconds.
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32
    1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64
    2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32
    3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64
    4 shortcut 1
    5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128
    6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
    7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    8 shortcut 5
    9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
   10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
   11 shortcut 8
   12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x  

---------yolo3camera_5.mp4 frames classified-----------
Average computation time per frame: 0.06913487994018923
Completed. Total time for computation: 5.0 minutes, 32.469637632369995 seconds.
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32
    1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64
    2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32
    3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64
    4 shortcut 1
    5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128
    6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
    7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    8 shortcut 5
    9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
   10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
   11 shortcut 8
   12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x 

In [ ]:
# Extract Frames of choice

frame_stream = cv2.VideoCapture("camera_6.mp4")
imgs = [2381, 4581, 3748, 3767, 3857, 4208]
for img in imgs:
    frame_stream.set(1, img)
    is_valid, frame = frame_stream.read()
    cv2.imwrite("camera6_"+str(img)+".png", frame)